# Interceptor

In this example, you gonna learn how to use `Interceptor` to capture intermediate values in the execution of a PyTorch module.

In [15]:
import torch # PyTorch

from pytorch_probing import Interceptor # Intercepts intermediate values.

We gonna start creating a example a module:

In [7]:
class ExampleModel(torch.nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, n_hidden=0):
        super().__init__()
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()

        if n_hidden > 0:
            layers = []
            for _ in range(n_hidden):
                layers.append(torch.nn.Linear(hidden_size, hidden_size))
                layers.append(torch.nn.ReLU())
            self.hidden_layers = torch.nn.Sequential(*layers)
        self._n_hidden = n_hidden

        self.linear2 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        if self._n_hidden > 0:
            x = self.hidden_layers(x)
        x = self.linear2(x)

        return x

In this case, we gonna use 2 hidden layers between the first and last layer:

In [8]:
input_size = 2
hidden_size = 3
output_size = 1
n_hidden = 2

model = ExampleModel(input_size, hidden_size, output_size, n_hidden)
model.eval()

ExampleModel(
  (linear1): Linear(in_features=2, out_features=3, bias=True)
  (relu): ReLU()
  (hidden_layers): Sequential(
    (0): Linear(in_features=3, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=3, bias=True)
    (3): ReLU()
  )
  (linear2): Linear(in_features=3, out_features=1, bias=True)
)

Than, we gonna pass our created module to a Interceptor, with the paths of the submodules we wanna get its outputs. Notices that we can use "." to get inner submodules:

In [9]:
paths = ["linear1", "hidden_layers.2", "linear2"]

intercepted_model = Interceptor(model, paths, detach=False)
intercepted_model.eval()

Interceptor(
  (_module): ExampleModel(
    (linear1): InterceptorLayer(
      (_module): Linear(in_features=2, out_features=3, bias=True)
    )
    (relu): ReLU()
    (hidden_layers): Sequential(
      (0): Linear(in_features=3, out_features=3, bias=True)
      (1): ReLU()
      (2): InterceptorLayer(
        (_module): Linear(in_features=3, out_features=3, bias=True)
      )
      (3): ReLU()
    )
    (linear2): InterceptorLayer(
      (_module): Linear(in_features=3, out_features=1, bias=True)
    )
  )
)

The Interceptor modifies in-place the original module:

In [10]:
model

ExampleModel(
  (linear1): InterceptorLayer(
    (_module): Linear(in_features=2, out_features=3, bias=True)
  )
  (relu): ReLU()
  (hidden_layers): Sequential(
    (0): Linear(in_features=3, out_features=3, bias=True)
    (1): ReLU()
    (2): InterceptorLayer(
      (_module): Linear(in_features=3, out_features=3, bias=True)
    )
    (3): ReLU()
  )
  (linear2): InterceptorLayer(
    (_module): Linear(in_features=3, out_features=1, bias=True)
  )
)

We pass a example input throught the model:

In [11]:
inputs = torch.randn([10, 2])

with torch.no_grad():
    outputs = intercepted_model(inputs)

In [13]:
outputs

tensor([[-0.1188],
        [-0.1146],
        [-0.1182],
        [-0.1188],
        [-0.1169],
        [-0.0354],
        [-0.0088],
        [-0.1188],
        [-0.0176],
        [-0.0311]])

And the interceptor captures the required outputs and stores then in the "outputs" attribute:

In [12]:
intercepted_model.outputs

{'linear1': tensor([[-2.0626e-01, -3.4000e-01, -7.1013e-01],
         [ 1.4269e-01,  4.0346e-01, -2.8771e-01],
         [ 1.2523e-03,  3.7725e-02, -4.4547e-01],
         [-9.4030e-02, -4.1919e-01, -5.0774e-01],
         [ 2.8304e-02,  1.4661e-01, -4.2343e-01],
         [ 4.4011e-01,  1.2838e+00,  2.0783e-02],
         [ 5.2744e-01,  1.4350e+00,  1.3378e-01],
         [-4.4810e-01, -1.0078e+00, -9.7100e-01],
         [ 5.7576e-01,  1.2486e+00,  2.5274e-01],
         [ 5.0058e-01,  1.2254e+00,  1.3309e-01]]),
 'hidden_layers.2': tensor([[-0.1331, -0.3591,  0.5654],
         [-0.1674, -0.3851,  0.5777],
         [-0.1380, -0.3628,  0.5672],
         [-0.1331, -0.3591,  0.5654],
         [-0.1489, -0.3711,  0.5711],
         [-0.3302, -0.4968,  0.8093],
         [-0.3599, -0.5250,  0.8870],
         [-0.1331, -0.3591,  0.5654],
         [-0.3189, -0.4983,  0.8614],
         [-0.3161, -0.4908,  0.8217]]),
 'linear2': tensor([[-0.1188],
         [-0.1146],
         [-0.1182],
         [-0.11

We can clean this outputs with the `interceptor_clear` method:

In [14]:
intercepted_model.interceptor_clear()
intercepted_model.outputs

{'linear1': None, 'hidden_layers.2': None, 'linear2': None}

And return the model to its original state with the reduce method:

In [16]:
intercepted_model.reduce()

model

ExampleModel(
  (linear1): Linear(in_features=2, out_features=3, bias=True)
  (relu): ReLU()
  (hidden_layers): Sequential(
    (0): Linear(in_features=3, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=3, bias=True)
    (3): ReLU()
  )
  (linear2): Linear(in_features=3, out_features=1, bias=True)
)